## 단순한 계층 구현하기
### - 곱셈 계층 구현하기
- 사과 쇼핑
- 곱셈 노드를 MulLayer, 덧셈 노드를 AddLayer로 구현
- forward()는 순전파, backward() 역전파

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x*y
        return out
    
    def backward(self, dout):
        dx = dout*self.y
        dy = dout*self.x
        
        return dx, dy

In [2]:
apple = 100
apple_num = 2
tax = 1.1

In [3]:
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

In [4]:
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

In [5]:
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

In [6]:
print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)

price: 220
dApple: 2.2
dApple_num: 110
dTax: 200


In [7]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x+y
        return out
    
    def backward(self, dout):
        dx = dout*1
        dy = dout*1
        return dx, dy

In [8]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

In [9]:
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

In [10]:
apple_price = mul_apple_layer.forward(apple, apple_num)  
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

In [12]:
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

In [13]:
print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))

print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


## 활성화 함수 구현


### - ReLU 계층 구현하기

In [14]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0
        
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

In [16]:
import numpy as np
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [17]:
mask = (x<=0)
print(mask)

[[False  True]
 [ True False]]


### - sigmoid 계층 구현하기

In [ ]:
class Sigmoid:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        out = 1/(1+np.exp(-x))
        self.out = out
        return out
        
    def backward(self, dout):
        dx = dout*(1.0 - self.out)*self.out
        return dx

## 출력 함수 구현

In [18]:
X = np.random.rand(2)
W = np.random.rand(2,3)
B = np.random.rand(3)

print(X.shape)
print(W.shape)
print(B.shape)

(2,)
(2, 3)
(3,)


In [ ]:
Y = np.dot(X, W) + B
print(Y)

In [23]:
X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1,2,3])

print("X_dot_W\n",X_dot_W)
print("X_dot_W + B \n", X_dot_W + B)

X_dot_W
 [[ 0  0  0]
 [10 10 10]]
X_dot_W + B 
 [[ 1  2  3]
 [11 12 13]]


In [22]:
dY = np.array([[1,2,3], [4,5,6]])
print("dY\n", dY)

dB = np.sum(dY, axis=0)
print("dB\n", dB)

dY
 [[1 2 3]
 [4 5 6]]
dB
 [5 7 9]


In [ ]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, out):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

In [24]:
def softmax(matrix):
    maximum_of_matrix = np.max(matrix)
    difference_from_maximum = matrix - maximum_of_matrix
    exponential_of_difference = np.exp(difference_from_maximum)
    sum_of_exponential = np.sum(exponential_of_difference)
    y = exponential_of_difference / sum_of_exponential
    return y

def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t*np.log(y+delta))

class softmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size

## 최종 구현

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)

import sys, os
sys.path.append('./mnist_dataset')
import numpy as np
import pickle
from mnist_dataset.mnist import load_mnist
import matplotlib.pylab as plt

In [ ]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True,
                                                  one_hot_label=True)

In [ ]:
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

In [ ]:
x_batch = x_train[:3]
t_batch = t_train[:3]

In [ ]:
grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)